Loading Data

In [0]:
'''
Mustafa Aslanov HW8
'''

'\nMustafa Aslanov HW8\n'

In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt 
import keras 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
from PIL import Image
import re

Using TensorFlow backend.


In [0]:
from keras.utils import to_categorical 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Dense, Dropout 
from keras.layers import Flatten, BatchNormalization

# Handchars

In [0]:
train_150 = pd.read_csv('HandChars_150_Train.txt', sep='#', engine='python')
test_50 = pd.read_csv('HandChars_50_Test.txt', sep='#', engine='python')

train_labels_150 = train_150.PatN
train_type_150 = train_150.PatType
train_prob_150 = train_150.PatProb
train_height_150 = train_150.SizeH
train_width_150 = train_150.SizeW
train_data_150 = train_150.Data

test_labels_50 = test_50.PatN
test_type_50 = test_50.PatType
test_prob_50 = test_50.PatProb
test_height_50 = test_50.SizeH
test_width_50 = test_50.SizeW
test_data_50 = test_50.Data

In [0]:
train_float_150=[]
for i in range (0,train_data_150.shape[0]):
  train_float_150.append(list(train_data_150[i]))

test_float_50=[]
for i in range (0,test_data_50.shape[0]):
  test_float_50.append(list(test_data_50[i]))

In [0]:
train_float_150 = pd.DataFrame(train_float_150) 
test_float_50 = pd.DataFrame(test_float_50) 

In [0]:
train_float_150 = train_float_150.replace({None: 0})
test_float_50 = test_float_50.replace({None: 0}) 

In [0]:
traindata_150= np.array(train_float_150)
testdata_50 = np.array(test_float_50) 

In [0]:
# adding padding to make them all same size
traindata_150 = np.pad(traindata_150,((0,0),(188,188)),'constant')
testdata_50 = np.pad(testdata_50,((0,0),(3,3)),'constant') #72x71

## MLP for Handchars


In [0]:
mlp_150 = MLPClassifier(hidden_layer_sizes=(120,120,120),max_iter= 400)

In [0]:
mlp_150.fit(traindata_150, train_labels_150)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(120, 120, 120), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
predictions_150 = mlp_150.predict(testdata_50)

In [0]:
print(classification_report(test_labels_50, predictions_150))

              precision    recall  f1-score   support

           0       0.43      0.46      0.45        50
           1       0.38      0.36      0.37        50
           2       0.36      0.38      0.37        50
           3       0.32      0.34      0.33        50
           4       0.26      0.22      0.24        50
           5       0.44      0.58      0.50        50
           6       0.38      0.34      0.36        50
           7       0.44      0.30      0.36        50
           8       0.31      0.34      0.33        50
           9       0.38      0.30      0.33        50
          10       0.53      0.52      0.53        50
          11       0.46      0.38      0.42        50
          12       0.29      0.30      0.30        50
          13       0.61      0.74      0.67        50
          14       0.53      0.46      0.49        50
          15       0.37      0.46      0.41        50
          16       0.36      0.40      0.38        50
          17       0.53    

##CNN for Handchars


In [0]:
trainY = to_categorical(train_float_150.iloc[:, 0], num_classes=28) 
testY = to_categorical(test_float_50.iloc[:, 0], num_classes=28) 

In [27]:
trainY.shape

(4200, 28)

In [0]:
rows = 72
cols = 71

# reshaping data to fit in CNN 
traindata = traindata_150.reshape(traindata_150.shape[0], rows, cols, 1)
testdata = testdata_50.reshape(testdata_50.shape[0], rows, cols, 1)

# traindata = traindata.astype('float32')
# testdata = testdata.astype('float32')

In [0]:
def baseline_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(rows, cols, 1)))
    model.add(Conv2D(50, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(BatchNormalization())
    model.add(Conv2D(100, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(BatchNormalization())
    model.add(Conv2D(150, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(150, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(28, activation='softmax'))
    return model

In [0]:
model = baseline_model()
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [29]:
history = model.fit(traindata, trainY,
          batch_size=512,
          epochs=5,
          verbose=2,
          validation_data=(testdata, testY))

Train on 4200 samples, validate on 1400 samples
Epoch 1/5
 - 8s - loss: 0.1188 - accuracy: 0.9800 - val_loss: 1.1086 - val_accuracy: 0.9857
Epoch 2/5
 - 8s - loss: 0.1133 - accuracy: 0.9826 - val_loss: 0.4862 - val_accuracy: 0.9857
Epoch 3/5
 - 8s - loss: 0.1328 - accuracy: 0.9800 - val_loss: 0.5248 - val_accuracy: 0.9857
Epoch 4/5
 - 8s - loss: 0.1080 - accuracy: 0.9814 - val_loss: 0.4248 - val_accuracy: 0.9857
Epoch 5/5
 - 8s - loss: 0.1101 - accuracy: 0.9821 - val_loss: 0.2457 - val_accuracy: 0.9857


#Handchars 32

In [0]:
train32_150 = pd.read_csv('HandChars32_150_Train.txt', sep='#', engine='python')
test32_50 = pd.read_csv('HandChars32_50_Test.txt', sep='#', engine='python')

train_labels32_150 = train32_150.PatN
train_type32_150 = train32_150.PatType
train_prob32_150 = train32_150.PatProb
train_height32_150 = 32
train_width32_150 = 32
train_data32_150 = train32_150.Data

test_labels32_50 = test32_50.PatN
test_type32_50 = test32_50.PatType
test_prob32_50 = test32_50.PatProb
test_height32_50 = 32
test_width32_50 = 32
test_data32_50 = test32_50.Data

In [0]:
train_float32=[]
for i in range (0,train_data32_150.shape[0]):
  train_float32.append(list(train_data32_150[i]))

test_float32=[]
for i in range (0,test_data32_50.shape[0]):
  test_float32.append(list(test_data32_50[i]))

In [0]:
train_float32 = pd.DataFrame(train_float32) 
test_float32 = pd.DataFrame(test_float32) 

## MLP for Handchars 32

In [0]:
mlp32 = MLPClassifier(hidden_layer_sizes=(120,120,120),max_iter= 400)

In [0]:
mlp32.fit(train_float32, train_labels32_150)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(120, 120, 120), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
predictions32 = mlp32.predict(test_float32)

In [0]:
print(classification_report(test_labels32_50, predictions32))

              precision    recall  f1-score   support

           0       0.62      0.62      0.62        50
           1       0.62      0.80      0.70        50
           2       0.69      0.92      0.79        50
           3       0.88      0.70      0.78        50
           4       0.58      0.62      0.60        50
           5       0.79      0.90      0.84        50
           6       0.67      0.90      0.77        50
           7       0.85      0.68      0.76        50
           8       0.87      0.78      0.82        50
           9       0.75      0.76      0.75        50
          10       0.92      0.48      0.63        50
          11       0.70      0.86      0.77        50
          12       0.65      0.66      0.65        50
          13       0.79      0.62      0.70        50
          14       0.48      0.74      0.58        50
          15       0.70      0.76      0.73        50
          16       0.85      0.58      0.69        50
          17       0.92    

##CNN for Handchars 32

In [0]:
traindata32= np.array(train_float32)
testdata32 = np.array(test_float32) 

In [0]:
trainY32 = to_categorical(train_labels32_150) 
testY32 = to_categorical(test_labels32_50) 

In [0]:
rows32 = 32
cols32 = 32

# reshaping data to fit in CNN 
traindata32 = traindata32.reshape(train_data32_150.shape[0], rows32, cols32, 1)
testdata32 = testdata32.reshape(test_data32_50.shape[0], rows32, cols32, 1)

# traindata32 = traindata32.astype('float32')
# testdata32 = testdata32.astype('float32')

In [0]:
def baseline_model32():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(rows32, cols32, 1)))
    model.add(Conv2D(50, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(BatchNormalization())
    model.add(Conv2D(100, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(BatchNormalization())
    model.add(Conv2D(150, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(150, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(28, activation='softmax'))
    return model

In [0]:
model32 = baseline_model32()
model32.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [0]:
history32 = model32.fit(traindata32, trainY32,
          batch_size=512,
          epochs=20,
          verbose=2,
          validation_data=(testdata32, testY32))

Train on 4200 samples, validate on 1400 samples
Epoch 1/20
 - 1s - loss: 0.0913 - accuracy: 0.9695 - val_loss: 0.8933 - val_accuracy: 0.8071
Epoch 2/20
 - 1s - loss: 0.1000 - accuracy: 0.9681 - val_loss: 1.0197 - val_accuracy: 0.7600
Epoch 3/20
 - 1s - loss: 0.0772 - accuracy: 0.9738 - val_loss: 1.0755 - val_accuracy: 0.7614
Epoch 4/20
 - 1s - loss: 0.0678 - accuracy: 0.9764 - val_loss: 1.1624 - val_accuracy: 0.7650
Epoch 5/20
 - 1s - loss: 0.0741 - accuracy: 0.9738 - val_loss: 0.7469 - val_accuracy: 0.8379
Epoch 6/20
 - 1s - loss: 0.0775 - accuracy: 0.9693 - val_loss: 0.9245 - val_accuracy: 0.8243
Epoch 7/20
 - 1s - loss: 0.0716 - accuracy: 0.9750 - val_loss: 0.9778 - val_accuracy: 0.8121
Epoch 8/20
 - 1s - loss: 0.0609 - accuracy: 0.9795 - val_loss: 0.8479 - val_accuracy: 0.8314
Epoch 9/20
 - 1s - loss: 0.0770 - accuracy: 0.9721 - val_loss: 0.8274 - val_accuracy: 0.8386
Epoch 10/20
 - 1s - loss: 0.0715 - accuracy: 0.9757 - val_loss: 0.9312 - val_accuracy: 0.8279
Epoch 11/20
 - 1s - l

#PDC calculated handchars

In [0]:
train_pdc = pd.read_csv('HandChars_FeatExt_PDC4428_150_Train.txt', sep='#', engine='python')
test_pdc = pd.read_csv('HandChars_FeatExt_PDC4428_50_Test.txt', sep='#', engine='python')

train_labels_pdc = train_pdc.PatN
train_type_pdc   = train_pdc.PatType
train_prob_pdc   = train_pdc.PatProb
train_height_pdc = train_pdc.SizeH
train_width_pdc = train_pdc.SizeW
train_data_pdc  = train_pdc.Data

test_labels_pdc = test_pdc.PatN
test_type_pdc = test_pdc.PatType
test_prob_pdc = test_pdc.PatProb
test_height_pdc = test_pdc.SizeH
test_width_pdc = test_pdc.SizeW
test_data_pdc = test_pdc.Data

In [0]:
traindata_pdc = []
for row in train_data_pdc:
  row = re.sub(r',', '.', row)
  row = row.split(';')
  traindata_pdc.append(row)

testdata_pdc = []
for row in test_data_pdc:
  row = re.sub(r',', '.', row)
  row = row.split(';')
  testdata_pdc.append(row)

In [0]:
traindata_pdc = pd.DataFrame(traindata_pdc) 
testdata_pdc = pd.DataFrame(testdata_pdc) 

In [0]:
traindataar_pdc= (np.array(traindata_pdc)).astype("float32")
testdataar_pdc = (np.array(testdata_pdc)).astype("float32") 

##MLP for PDC calculated handchars

In [0]:
mlp_pdc = MLPClassifier(hidden_layer_sizes=(120,120,120),max_iter= 400)

In [0]:
mlp_pdc.fit(traindata_pdc, train_labels_pdc)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(120, 120, 120), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
predictions_pdc = mlp_pdc.predict(testdata_pdc)

In [0]:
print(classification_report(test_labels_pdc, predictions_pdc))

              precision    recall  f1-score   support

           0       0.78      0.94      0.85        50
           1       0.84      0.94      0.89        50
           2       0.82      0.94      0.88        50
           3       0.90      0.72      0.80        50
           4       0.88      0.58      0.70        50
           5       0.67      0.94      0.78        50
           6       0.69      0.96      0.80        50
           7       0.86      0.62      0.72        50
           8       0.83      0.90      0.87        50
           9       0.82      0.82      0.82        50
          10       0.84      0.76      0.80        50
          11       0.67      0.86      0.75        50
          12       0.82      0.80      0.81        50
          13       0.95      0.78      0.86        50
          14       0.80      0.80      0.80        50
          15       0.85      0.90      0.87        50
          16       1.00      0.82      0.90        50
          17       0.94    

##CNN for PDC calculated handchars

In [0]:
trainYpdc = to_categorical(train_labels_pdc) 
testYpdc = to_categorical(test_labels_pdc) 

In [0]:
rowspdc = 16
colspdc = 16

# reshaping data to fit in CNN 
traindatapdc = traindataar_pdc.reshape(traindataar_pdc.shape[0], rowspdc, colspdc, 1)
testdatapdc = testdataar_pdc.reshape(testdataar_pdc.shape[0], rowspdc, colspdc, 1)

traindatapdc = traindatapdc.astype('float32')
testdatapdc  = testdatapdc.astype('float32')

In [0]:
def baseline_modelpdc():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(rowspdc, colspdc, 1)))
    model.add(Conv2D(50, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(BatchNormalization())
    model.add(Conv2D(100, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(BatchNormalization())
    model.add(Conv2D(150, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(150, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(28, activation='softmax'))
    return model

In [0]:
modelpdc = baseline_modelpdc()
modelpdc.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
#adadelta = 42.7%, adagrad and adam = 9%, rmsprop and sgd = 17%

In [0]:
history = modelpdc.fit(traindatapdc, trainYpdc,
          batch_size=512,
          epochs=20,
          verbose=2,
          validation_data=(testdatapdc, testYpdc))

Train on 4200 samples, validate on 1400 samples
Epoch 1/20
 - 0s - loss: 0.1139 - accuracy: 0.9612 - val_loss: 1.1425 - val_accuracy: 0.7714
Epoch 2/20
 - 0s - loss: 0.1155 - accuracy: 0.9636 - val_loss: 1.0801 - val_accuracy: 0.8064
Epoch 3/20
 - 0s - loss: 0.1088 - accuracy: 0.9648 - val_loss: 1.0737 - val_accuracy: 0.7936
Epoch 4/20
 - 0s - loss: 0.1034 - accuracy: 0.9636 - val_loss: 0.9641 - val_accuracy: 0.8143
Epoch 5/20
 - 0s - loss: 0.1116 - accuracy: 0.9636 - val_loss: 0.9321 - val_accuracy: 0.8221
Epoch 6/20
 - 0s - loss: 0.1085 - accuracy: 0.9679 - val_loss: 0.8894 - val_accuracy: 0.8207
Epoch 7/20
 - 0s - loss: 0.0968 - accuracy: 0.9671 - val_loss: 0.9618 - val_accuracy: 0.8064
Epoch 8/20
 - 0s - loss: 0.0942 - accuracy: 0.9657 - val_loss: 1.0165 - val_accuracy: 0.8136
Epoch 9/20
 - 0s - loss: 0.1053 - accuracy: 0.9617 - val_loss: 1.0968 - val_accuracy: 0.7864
Epoch 10/20
 - 0s - loss: 0.0896 - accuracy: 0.9702 - val_loss: 0.9415 - val_accuracy: 0.8229
Epoch 11/20
 - 0s - l

#Extras

##For Handchars

In [0]:
rows = 72
cols = 71
traindata_extra = traindata_150.reshape(traindata_150.shape[0], rows, cols)
testdata_extra = testdata_50.reshape(testdata_50.shape[0], rows, cols)

In [0]:
rgb_train_extra = np.repeat(traindata_extra[..., np.newaxis], 3, -1)
rgb_test_extra = np.repeat(testdata_extra[..., np.newaxis], 3, -1)

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(rows,cols,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  import sys


In [0]:
input_shape=(rows,cols,3)
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model_extra = Sequential()
model_extra.add(restnet)
model_extra.add(Dense(512, activation='relu', input_dim=input_shape))
model_extra.add(Dropout(0.3))
model_extra.add(Dense(512, activation='relu'))
model_extra.add(Dropout(0.3))
model_extra.add(Dense(28, activation='sigmoid'))
model_extra.compile(loss='binary_crossentropy',
              optimizer=optimizers.rmsprop(learning_rate=2e-5),
              metrics=['accuracy'])

In [0]:
history = model_extra.fit(rgb_train_extra, trainY,
                              steps_per_epoch=1, 
                              epochs=20,
                              verbose=2)

Epoch 1/20
 - 21s - loss: 0.9700 - accuracy: 0.5094
Epoch 2/20
 - 7s - loss: 0.7339 - accuracy: 0.6159
Epoch 3/20
 - 7s - loss: 0.6282 - accuracy: 0.6785
Epoch 4/20
 - 7s - loss: 0.5605 - accuracy: 0.7219
Epoch 5/20
 - 7s - loss: 0.5092 - accuracy: 0.7582
Epoch 6/20
 - 7s - loss: 0.4717 - accuracy: 0.7842
Epoch 7/20
 - 7s - loss: 0.4402 - accuracy: 0.8057
Epoch 8/20
 - 7s - loss: 0.4129 - accuracy: 0.8230
Epoch 9/20
 - 7s - loss: 0.3916 - accuracy: 0.8385
Epoch 10/20
 - 7s - loss: 0.3728 - accuracy: 0.8528
Epoch 11/20
 - 7s - loss: 0.3602 - accuracy: 0.8600
Epoch 12/20
 - 7s - loss: 0.3451 - accuracy: 0.8702
Epoch 13/20
 - 7s - loss: 0.3331 - accuracy: 0.8780
Epoch 14/20
 - 7s - loss: 0.3234 - accuracy: 0.8862
Epoch 15/20
 - 7s - loss: 0.3138 - accuracy: 0.8920
Epoch 16/20
 - 7s - loss: 0.3053 - accuracy: 0.8974
Epoch 17/20
 - 7s - loss: 0.2979 - accuracy: 0.9022
Epoch 18/20
 - 7s - loss: 0.2948 - accuracy: 0.9056
Epoch 19/20
 - 7s - loss: 0.2868 - accuracy: 0.9098
Epoch 20/20
 - 7s - 

##For Handchars 32

In [0]:
traindata32_extra = traindata32.reshape(traindata32.shape[0], rows32, cols32)
testdata32_extra = testdata32.reshape(testdata32.shape[0], rows32, cols32)

In [0]:
rgb_train32_extra = np.repeat(traindata32_extra[..., np.newaxis], 3, -1)
rgb_test32_extra = np.repeat(testdata32_extra[..., np.newaxis], 3, -1

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(32,32,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  import sys


In [0]:
input_shape=(32,32,3)
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model32_extra = Sequential()
model32_extra.add(restnet)
model32_extra.add(Dense(512, activation='relu', input_dim=input_shape))
model32_extra.add(Dropout(0.3))
model32_extra.add(Dense(512, activation='relu'))
model32_extra.add(Dropout(0.3))
model32_extra.add(Dense(28, activation='sigmoid'))
model32_extra.compile(loss='binary_crossentropy',
              optimizer=optimizers.rmsprop(learning_rate=2e-5),
              metrics=['accuracy'])

In [0]:
history = model32_extra.fit(rgb_train32_extra, trainY32,
                              steps_per_epoch=1, 
                              epochs=20,
                              verbose=2)

Epoch 1/20
 - 2s - loss: 1.0225 - accuracy: 0.4749
Epoch 2/20
 - 1s - loss: 0.9591 - accuracy: 0.5005
Epoch 3/20
 - 1s - loss: 0.9144 - accuracy: 0.5224
Epoch 4/20
 - 1s - loss: 0.8820 - accuracy: 0.5363
Epoch 5/20
 - 1s - loss: 0.8554 - accuracy: 0.5482
Epoch 6/20
 - 1s - loss: 0.8344 - accuracy: 0.5603
Epoch 7/20
 - 1s - loss: 0.8086 - accuracy: 0.5716
Epoch 8/20
 - 1s - loss: 0.7871 - accuracy: 0.5841
Epoch 9/20
 - 1s - loss: 0.7707 - accuracy: 0.5957
Epoch 10/20
 - 1s - loss: 0.7565 - accuracy: 0.6015
Epoch 11/20
 - 1s - loss: 0.7390 - accuracy: 0.6114
Epoch 12/20
 - 1s - loss: 0.7242 - accuracy: 0.6198
Epoch 13/20
 - 1s - loss: 0.7097 - accuracy: 0.6298
Epoch 14/20
 - 1s - loss: 0.6964 - accuracy: 0.6376
Epoch 15/20
 - 1s - loss: 0.6775 - accuracy: 0.6497
Epoch 16/20
 - 1s - loss: 0.6652 - accuracy: 0.6570
Epoch 17/20
 - 1s - loss: 0.6530 - accuracy: 0.6633
Epoch 18/20
 - 1s - loss: 0.6432 - accuracy: 0.6716
Epoch 19/20
 - 1s - loss: 0.6285 - accuracy: 0.6792
Epoch 20/20
 - 1s - l

##For PDC

In [0]:
rowspdc = 16
colspdc = 16
traindata_pdc_extra = traindataar_pdc.reshape(traindataar_pdc.shape[0], rowspdc, colspdc)
testdata_pdc_extra = testdataar_pdc.reshape(testdataar_pdc.shape[0], rowspdc, colspdc)

In [0]:
rgb_train_pdc_extra = np.repeat(traindata_pdc_extra[..., np.newaxis], 3, -1)
rgb_test_pdc_extra = np.repeat(testdata_pdc_extra[..., np.newaxis], 3, -1)


In [0]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(32,32,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  import sys


In [0]:
input_shape=(16,16,3)
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model_pdc_extra = Sequential()
model_pdc_extra.add(vg)
model_pdc_extra.add(Dense(512, activation='relu', input_dim=input_shape))
model_pdc_extra.add(Dropout(0.3))
model_pdc_extra.add(Dense(512, activation='relu'))
model_pdc_extra.add(Dropout(0.3))
model_pdc_extra.add(Dense(28, activation='sigmoid'))
model_pdc_extra.compile(loss='binary_crossentropy',
              optimizer=optimizers.rmsprop(learning_rate=2e-5),
              metrics=['accuracy'])

In [0]:
history = model_pdc_extra.fit(rgb_train_pdc_extra, trainYpdc,
                              steps_per_epoch=1, 
                              epochs=20,
                              verbose=2)

Epoch 1/20
 - 21s - loss: 0.9700 - accuracy: 0.5094
Epoch 2/20
 - 7s - loss: 0.7339 - accuracy: 0.6159
Epoch 3/20
 - 7s - loss: 0.6282 - accuracy: 0.6785
Epoch 4/20
 - 7s - loss: 0.5605 - accuracy: 0.7219
Epoch 5/20
 - 7s - loss: 0.5092 - accuracy: 0.7582
Epoch 6/20
 - 7s - loss: 0.4717 - accuracy: 0.7842
Epoch 7/20
 - 7s - loss: 0.4402 - accuracy: 0.8057
Epoch 8/20
 - 7s - loss: 0.4129 - accuracy: 0.8230
Epoch 9/20
 - 7s - loss: 0.3916 - accuracy: 0.8385
Epoch 10/20
 - 7s - loss: 0.3728 - accuracy: 0.8528
Epoch 11/20
 - 7s - loss: 0.3602 - accuracy: 0.8600
Epoch 12/20
 - 7s - loss: 0.3451 - accuracy: 0.8702
Epoch 13/20
 - 7s - loss: 0.3331 - accuracy: 0.8780
Epoch 14/20
 - 7s - loss: 0.3234 - accuracy: 0.8862
Epoch 15/20
 - 7s - loss: 0.3138 - accuracy: 0.8920
Epoch 16/20
 - 7s - loss: 0.3053 - accuracy: 0.8974
Epoch 17/20
 - 7s - loss: 0.2979 - accuracy: 0.9022
Epoch 18/20
 - 7s - loss: 0.2948 - accuracy: 0.9056
Epoch 19/20
 - 7s - loss: 0.2868 - accuracy: 0.9098
Epoch 20/20
 - 7s - 